# REQUIRES LARGE/GPU INSTANCE TYPE
## google/flan-t5-base toxicity score:
Mean: 0.11131308024933918 - Std: 0.26920890184712915

In [4]:
import psutil

notebook_memory = psutil.virtual_memory()
print(notebook_memory)

if notebook_memory.total < 32 * 1000 * 1000 * 1000:
    print('*******************************************')    
    print('YOU ARE NOT USING THE CORRECT INSTANCE TYPE')
    print('PLEASE CHANGE INSTANCE TYPE TO  m5.2xlarge ')
    print('*******************************************')
else:
    correct_instance_type=True

svmem(total=16610865152, available=12604526592, percent=24.1, used=3704487936, free=810323968, active=5607727104, inactive=9077927936, buffers=2498560, cached=12093554688, shared=15458304, slab=641945600)
*******************************************
YOU ARE NOT USING THE CORRECT INSTANCE TYPE
PLEASE CHANGE INSTANCE TYPE TO  m5.2xlarge 
*******************************************


In [3]:
%store -r peft_fine_tuned_model_checkpoint

no stored variable or alias peft_fine_tuned_model_checkpoint


In [4]:
try:
    supervised_fine_tuned_model_checkpoint
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] Please run the notebooks in the PREPARE section before you continue.")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [5]:
print(peft_fine_tuned_model_checkpoint)

./tmp_models/google/flan-t5-base-supervised-fine-tuned/


# Quantitative Results with ROGUE Metric

The [ROUGE metric](https://en.wikipedia.org/wiki/ROUGE_(metric)) helps quantify the validity of summarizations produced by models. It compares summarizations to a "baseline" summary which is usually created by a human. While not perfect, it does give an indication to the overall increase in summarization effectiveness that we have accomplished by fine-tuning.

In [6]:
# %pip install --disable-pip-version-check -q \
#     transformers==4.27.2 \
#     datasets==2.9.0 \
#     accelerate==0.17.0 \
#     promptsource==0.2.3 \
#     evaluate==0.4.0 \
#     trl==0.4.1 \
#     rouge_score==0.1.2 \
#     loralib==0.1.1

Note: you may need to restart the kernel to use updated packages.


In [8]:
import argparse
import csv

import evaluate
import numpy as np
import torch
from datasets import load_dataset
from tqdm import tqdm
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

toxicity = evaluate.load("ybelkada/toxicity", "DaNLP/da-electra-hatespeech-detection", module_type="measurement")
ds = load_dataset("OxAISH-AL-LLM/wiki_toxic", split="test")

model_type = "all"
output_file = "toxicity.csv"
batch_size=64
num_samples=400
context_length=2000
max_new_tokens=30


if model_type == "all":
    MODELS_TO_TEST = [
        # "t5-base",        
        # "google/flan-t5-base",
        #supervised_fine_tuned_model_checkpoint,
        peft_fine_tuned_model_checkpoint,
    ]
    
NUM_SAMPLES = num_samples
BATCH_SIZE = batch_size
output_file = output_file
max_new_tokens = max_new_tokens
context_length = context_length
device = torch.cuda.current_device() if torch.cuda.is_available() else "cpu"

# consider only toxic prompts
ds = ds.filter(lambda x: x["label"] == 1)

toxicities = {}

# open a csv file
file = open(f"{output_file}", "w", newline="")
writer = csv.writer(file)
# add first rows
writer.writerow(["model_id", "mean_toxicity", "std_toxicity"])

for model_id in tqdm(MODELS_TO_TEST):
    model = AutoModelForSeq2SeqLM.from_pretrained(model_id, device_map='auto')
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "left"
    input_texts = []

    for i, example in enumerate(ds):
        # set seed
        torch.manual_seed(42)

        input_text = example["comment_text"]
        input_texts.append(input_text[:2000])

        if i > NUM_SAMPLES:
            break

        if (i + 1) % BATCH_SIZE == 0:
            inputs = tokenizer(input_texts, return_tensors="pt", padding=True).to(device)
            inputs.input_ids = inputs.input_ids[:context_length]
            inputs.attention_mask = inputs.attention_mask[:context_length]
            outputs = model.generate(**inputs, do_sample=True, max_new_tokens=max_new_tokens, use_cache=True)
            generated_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)
            generated_texts = [
                generated_text.replace(input_texts[i], "") for i, generated_text in enumerate(generated_texts)
            ]
            toxicity_score = toxicity.compute(predictions=generated_texts)
            input_texts = []

            if model_id not in toxicities:
                toxicities[model_id] = []
            toxicities[model_id].extend(toxicity_score["toxicity"])

    # last batch
    inputs = tokenizer(input_texts, return_tensors="pt", padding=True).to(device)
    outputs = model.generate(**inputs, do_sample=True, max_new_tokens=200)
    generated_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    generated_texts = [generated_text.replace(input_texts[i], "") for i, generated_text in enumerate(generated_texts)]
    toxicity_score = toxicity.compute(predictions=generated_texts)
    toxicities[model_id].extend(toxicity_score["toxicity"])

    # compute mean & std using np
    mean = np.mean(toxicities[model_id])
    std = np.std(toxicities[model_id])

    # save to file
    writer.writerow([model_id, mean, std])

    # print
    print(f"Model: {model_id} - Mean: {mean} - Std: {std}")

    model = None
    torch.cuda.empty_cache()

# close file
file.close()

Found cached dataset wiki_toxic (/root/.cache/huggingface/datasets/OxAISH-AL-LLM___wiki_toxic/default/1.0.0/09a67129f85f67f22107b0190f7c32050ef0dce44afeedc6e3e0ab7ab3bd709c)
Loading cached processed dataset at /root/.cache/huggingface/datasets/OxAISH-AL-LLM___wiki_toxic/default/1.0.0/09a67129f85f67f22107b0190f7c32050ef0dce44afeedc6e3e0ab7ab3bd709c/cache-411a66332d7a5b58.arrow
  0%|          | 0/1 [00:04<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.74 GiB (GPU 0; 22.20 GiB total capacity; 12.11 GiB already allocated; 1.54 GiB free; 12.23 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

# Release Resources

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>